In [1]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
#data_path = "\\users\\user\\onedrive\\a7\\Data"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 6, 23)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-06-23
yesterday: 2023-06-22


#### Restart and Run All

In [3]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [4]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1269250,TFFIF,2023-06-22,2023-06-22,0,7.75,7.60,-0.15,-1.94%,-3,0,7.90,7.55,"46,093,888",2023-06-22,2023-06-22,1


In [5]:
names = tmp["name"]
type(names)

pandas.core.series.Series

In [6]:
name = names.to_string(index=False)
type(name)

str

In [7]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'TFFIF'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9739,TFFIF,8.50,7.00,O,7.65,7.70,24.39,0.41,7.65,"10,000",-4,4,0,0,0,DOS,SET


### End of Tables in the process

In [8]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2023-06-22'
ORDER BY a.name



,name
0,AH
1,AWC
2,BANPU
3,BBL
4,BCH
5,BDMS
6,BEM
7,BH
8,CPNREIT
9,EA


In [9]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-06-22' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [10]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,BANPU,2023-06-22,2023-06-22,9.05,9.10,"301,185,316",9.15,8.95,B,SET50,0.05,0.55%
1,CPNREIT,2023-06-22,2023-06-22,10.10,10.40,"144,440,040",13.00,10.10,B,SET,0.30,2.97%
2,JMART,2023-06-15,2023-06-22,19.30,16.10,"574,772,464",20.10,16.10,B,SET50,-3.20,-16.58%
3,JMT,2023-06-21,2023-06-22,40.00,37.00,"95,821,884",43.75,37.00,B,SET50,-3.00,-7.50%
4,MCS,2023-06-14,2023-06-22,7.10,6.75,"14,775,280",7.15,6.75,B,SET,-0.35,-4.93%
5,SINGER,2023-06-15,2023-06-22,10.90,9.95,"313,931,200",11.30,9.95,B,SET100,-0.95,-8.72%
6,STA,2023-06-19,2023-06-22,19.00,17.60,"161,912,304",19.00,17.60,B,SET100,-1.40,-7.37%
7,SYNEX,2023-06-16,2023-06-22,13.10,12.70,"4,635,424",13.20,12.70,B,SET,-0.40,-3.05%
8,TMT,2023-06-21,2023-06-22,7.50,7.35,"491,420",7.50,7.35,B,SET,-0.15,-2.00%
9,AH,2023-06-22,2023-06-22,32.75,33.25,"19,039,072",35.00,32.75,I,SET,0.50,1.53%


In [11]:
df.shape

(34, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [12]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,BANPU,2023-06-22,2023-06-22,9.05,9.10,"301,185,316",9.15,8.95,0.55%,B
1,CPNREIT,2023-06-22,2023-06-22,10.10,10.40,"144,440,040",13.00,10.10,2.97%,B
2,JMART,2023-06-15,2023-06-22,19.30,16.10,"574,772,464",20.10,16.10,-16.58%,B
3,JMT,2023-06-21,2023-06-22,40.00,37.00,"95,821,884",43.75,37.00,-7.50%,B
4,MCS,2023-06-14,2023-06-22,7.10,6.75,"14,775,280",7.15,6.75,-4.93%,B
5,SINGER,2023-06-15,2023-06-22,10.90,9.95,"313,931,200",11.30,9.95,-8.72%,B
6,STA,2023-06-19,2023-06-22,19.00,17.60,"161,912,304",19.00,17.60,-7.37%,B
7,SYNEX,2023-06-16,2023-06-22,13.10,12.70,"4,635,424",13.20,12.70,-3.05%,B
8,TMT,2023-06-21,2023-06-22,7.50,7.35,"491,420",7.50,7.35,-2.00%,B
9,AH,2023-06-22,2023-06-22,32.75,33.25,"19,039,072",35.00,32.75,1.53%,I


In [13]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [14]:
df[cols].shape

(34, 10)

In [15]:
sales = df[cols]
sales

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,BANPU,2023-06-22,2023-06-22,9.05,9.10,301185316,9.15,8.95,0.55,B
1,CPNREIT,2023-06-22,2023-06-22,10.10,10.40,144440040,13.00,10.10,2.97,B
2,JMART,2023-06-15,2023-06-22,19.30,16.10,574772464,20.10,16.10,-16.58,B
3,JMT,2023-06-21,2023-06-22,40.00,37.00,95821884,43.75,37.00,-7.50,B
4,MCS,2023-06-14,2023-06-22,7.10,6.75,14775280,7.15,6.75,-4.93,B
5,SINGER,2023-06-15,2023-06-22,10.90,9.95,313931200,11.30,9.95,-8.72,B
6,STA,2023-06-19,2023-06-22,19.00,17.60,161912304,19.00,17.60,-7.37,B
7,SYNEX,2023-06-16,2023-06-22,13.10,12.70,4635424,13.20,12.70,-3.05,B
8,TMT,2023-06-21,2023-06-22,7.50,7.35,491420,7.50,7.35,-2.00,B
9,AH,2023-06-22,2023-06-22,32.75,33.25,19039072,35.00,32.75,1.53,I


In [16]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
prices = pd.read_csv(input_file)
prices

,name,date,price,change,%change,open,high,low,volume,value(m)
0,ACE,2023-06-23,1.89,-0.02,-1.047120,1.91,1.92,1.88,15610982,29631.01586
1,ADVANC,2023-06-23,211.00,-1.00,-0.471698,211.00,213.00,209.00,4423809,933654.78400
2,AEONTS,2023-06-23,186.00,-4.00,-2.105263,190.00,190.50,185.50,213604,40005.98350
3,AH,2023-06-23,32.25,-1.00,-3.007519,33.00,33.00,32.25,2130807,69306.26100
4,AIE,2023-06-23,1.44,0.01,0.699301,1.42,1.44,1.42,212273,302.73027
...,...,...,...,...,...,...,...,...,...,...
187,WHAIR,2023-06-23,7.20,0.00,0.000000,7.25,7.25,7.15,271400,1949.18500
188,WHART,2023-06-23,10.60,0.10,0.952381,10.40,10.70,10.40,2918600,30662.58040
189,WHAUP,2023-06-23,3.74,-0.02,-0.531915,3.76,3.78,3.72,837232,3132.20272
190,WICE,2023-06-23,8.45,0.00,0.000000,8.45,8.55,8.35,937825,7918.56665


In [17]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,date,price,change,%change,open,high,low,volume,value(m)
0,BANPU,2023-06-22,2023-06-22,9.05,9.10,301185316,9.15,8.95,0.55,B,2023-06-23,9.15,0.05,0.549451,9.10,9.15,9.05,47159009,4.291027e+05
1,CPNREIT,2023-06-22,2023-06-22,10.10,10.40,144440040,13.00,10.10,2.97,B,2023-06-23,11.00,0.60,5.769231,10.40,11.00,10.20,4383653,4.714373e+04
2,JMART,2023-06-15,2023-06-22,19.30,16.10,574772464,20.10,16.10,-16.58,B,2023-06-23,16.00,-0.10,-0.621118,16.20,16.50,15.80,15215746,2.448863e+05
3,JMT,2023-06-21,2023-06-22,40.00,37.00,95821884,43.75,37.00,-7.50,B,2023-06-23,36.75,-0.25,-0.675676,37.00,37.50,36.25,10632690,3.917201e+05
4,MCS,2023-06-14,2023-06-22,7.10,6.75,14775280,7.15,6.75,-4.93,B,2023-06-23,6.75,0.00,0.000000,6.80,6.80,6.55,691459,4.623915e+03
5,SINGER,2023-06-15,2023-06-22,10.90,9.95,313931200,11.30,9.95,-8.72,B,2023-06-23,9.85,-0.10,-1.005025,9.90,10.10,9.75,5246769,5.191748e+04
6,STA,2023-06-19,2023-06-22,19.00,17.60,161912304,19.00,17.60,-7.37,B,2023-06-23,17.10,-0.50,-2.840909,17.60,17.60,17.00,5556163,9.583450e+04
7,SYNEX,2023-06-16,2023-06-22,13.10,12.70,4635424,13.20,12.70,-3.05,B,2023-06-23,12.60,-0.10,-0.787402,12.70,12.90,12.50,252861,3.215710e+03
8,TMT,2023-06-21,2023-06-22,7.50,7.35,491420,7.50,7.35,-2.00,B,2023-06-23,7.35,0.00,0.000000,7.35,7.40,7.25,166230,1.215851e+03
9,AH,2023-06-22,2023-06-22,32.75,33.25,19039072,35.00,32.75,1.53,I,2023-06-23,32.25,-1.00,-3.007519,33.00,33.00,32.25,2130807,6.930626e+04


In [18]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
colu

['name',
 'fm_date',
 'to_date',
 'fm_price',
 'to_price',
 'qty',
 'max_price',
 'min_price',
 'percent',
 'status',
 'price',
 'change',
 'volume',
 'date']

In [19]:
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,BANPU,2023-06-22,2023-06-22,9.05,9.10,301185316,9.15,8.95,0.55,B,9.15,0.05,47159009,2023-06-23
1,CPNREIT,2023-06-22,2023-06-22,10.10,10.40,144440040,13.00,10.10,2.97,B,11.00,0.60,4383653,2023-06-23
2,JMART,2023-06-15,2023-06-22,19.30,16.10,574772464,20.10,16.10,-16.58,B,16.00,-0.10,15215746,2023-06-23
3,JMT,2023-06-21,2023-06-22,40.00,37.00,95821884,43.75,37.00,-7.50,B,36.75,-0.25,10632690,2023-06-23
4,MCS,2023-06-14,2023-06-22,7.10,6.75,14775280,7.15,6.75,-4.93,B,6.75,0.00,691459,2023-06-23
5,SINGER,2023-06-15,2023-06-22,10.90,9.95,313931200,11.30,9.95,-8.72,B,9.85,-0.10,5246769,2023-06-23
6,STA,2023-06-19,2023-06-22,19.00,17.60,161912304,19.00,17.60,-7.37,B,17.10,-0.50,5556163,2023-06-23
7,SYNEX,2023-06-16,2023-06-22,13.10,12.70,4635424,13.20,12.70,-3.05,B,12.60,-0.10,252861,2023-06-23
8,TMT,2023-06-21,2023-06-22,7.50,7.35,491420,7.50,7.35,-2.00,B,7.35,0.00,166230,2023-06-23
9,AH,2023-06-22,2023-06-22,32.75,33.25,19039072,35.00,32.75,1.53,I,32.25,-1.00,2130807,2023-06-23


In [20]:
df_merge[colu].shape

(34, 14)

In [21]:
df_merge.shape

(34, 19)

In [22]:
file_name = "daily-sales-prices.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(data_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(one_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

In [23]:
### Call ruby ruby\daily-out-new.rb

In [24]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [25]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [26]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AH       2023-06-23 2023-06-23  33.25  32.25  -3.01%  19,039,072  35.00  32.25 I Insert
AWC      2023-06-21 2023-06-23   5.05   4.88  -3.37% 838,247,352   5.30   4.84 I Update
BANPU    2023-06-22 2023-06-23   9.05   9.15    1.1% 301,185,316   9.15   8.95 B Update
BBL      2023-06-20 2023-06-23 161.00 163.00   1.24% 103,950,872 164.50 160.50 O Update
BCH      2023-06-23 2023-06-23  17.70  17.60  -0.56%  90,479,986  18.50  17.60 I Insert
BDMS     2023-06-22 2023-06-23  28.75  28.25  -1.74% 544,071,724  29.00  28.25 O Update
BEM      2023-06-21 2023-06-23   8.45   8.30  -1.78% 243,507,944   8.55   8.30 O Update
BH       2023-06-23 2023-06-23 232.00 228.00  -1.72%   7,421,714 236.00 226.00 O Insert
CPNREIT  2023-06-22 2023-06-23  10.10  11.00   8.91% 144,440,040  13.00  10.10 B Update
EA       2023-06-23 2023-06-23  

In [27]:
os.chdir("C:\\users\\user\\onedrive\\a7\\Daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\Daily'

### cd\ruby\port_lite
### rails runner db/ins_sales.rb